In [13]:
%%javascript
function ClickConnect(){
  console.log("Working");
  document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [14]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

from ipyleaflet import Map
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import os

import geopandas as gpd
import matplotlib.pyplot as plt

import geoplot as gplt
from folium.raster_layers import ImageOverlay
import IPython.display as display

# 数据预处理

重点关注按地理位置划分的犯罪类型，以及不同时间段的犯罪频率。

In [15]:
df = pd.read_csv('./Crime_data_final.csv')

In [16]:
df_f = df
df_f['DATE'] = df_f['DATE'].astype(str)

df_f['YEAR'] = df_f['DATE'].str.split('-').str[0]
df_f['MONTH'] = df_f['DATE'].str.split('-').str[1]
df_f['DAY'] = df_f['DATE'].str.split('-').str[2]

df_f['HOUR'] = df_f['TIME'].str.split(':').str[0]
df_f['MINUTE'] = df_f['TIME'].str.split(':').str[1]
df_f['SECOND'] = df_f['TIME'].str.split(':').str[2]

df_f['YEAR'] = df_f['YEAR'].astype(int)
df_f['MONTH'] = df_f['MONTH'].astype(int)
df_f['DAY'] = df_f['DAY'].astype(int)

df_f['HOUR'] = df_f['HOUR'].astype(int)
df_f['MINUTE'] = df_f['MINUTE'].astype(int)
df_f['SECOND'] = df_f['SECOND'].astype(int)

In [17]:
df_f.head()

,Unnamed: 0,SHIFT,METHOD,OFFENSE,WARD,DISTRICT,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,...,OBJECTID,COUNT,DATE,TIME,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND
0,0,EVENING,OTHERS,THEFT/OTHER,8,7.0,Cluster 38,007409 3,7409.0,Precinct 116,...,237164351,1,2020-07-09,02:13:24,2020,7,9,2,13,24
1,1,MIDNIGHT,GUN,HOMICIDE,8,7.0,Cluster 37,007407 2,7407.0,Precinct 119,...,237164363,1,2020-05-19,04:00:00,2020,5,19,4,0,0
2,2,MIDNIGHT,GUN,HOMICIDE,8,7.0,Cluster 37,007407 2,7407.0,Precinct 119,...,237164364,1,2020-05-19,04:00:00,2020,5,19,4,0,0
3,3,EVENING,OTHERS,THEFT F/AUTO,2,2.0,Cluster 8,005802 5,5802.0,Precinct 129,...,237170410,1,2020-01-17,02:39:05,2020,1,17,2,39,5
4,4,MIDNIGHT,OTHERS,THEFT F/AUTO,2,3.0,Cluster 7,005001 2,5001.0,Precinct 16,...,237170411,1,2020-01-28,05:14:39,2020,1,28,5,14,39


# 自适应核密度分析

In [18]:
df_f2 = df_f[['LONGITUDE', 'LATITUDE']].reset_index()
df_f2.rename(columns = {'index':'ID', 'LONGITUDE':'x', 'LATITUDE':'y'}, inplace = True)
df_f2.to_csv('./crime_data_2.csv', index=False)

In [19]:
df_f2

,ID,x,y
0,0,-76.978919,38.843884
1,1,-76.993181,38.862253
2,2,-76.993181,38.862253
3,3,-77.023986,38.901478
4,4,-77.030785,38.911868
...,...,...,...
27877,27877,-77.019900,38.904903
27878,27878,-77.036506,38.915931
27879,27879,-77.044226,38.923239
27880,27880,-77.009083,38.873808


In [20]:
# 导入所需的库
import numpy as np
import math
import matplotlib.pyplot as plt
import time

# 定义自适应KDE函数
def adaptiveKDE(input_dir, result_dir):
    # 读取输入文件中的所有行
    lines = open(input_dir).readlines()
    # 创建或打开结果文件用于写入
    result = open(result_dir, 'w+')
    # 写入结果文件的标题行
    result.write('ID,x,y,h\n')
    # 记录开始时间
    time_start = time.time()
    
    # 初始化参数
    a, delta_a, e_a = 0.5, 0.1, 0.005
    # 初始化点的集合和数组
    points = {}
    x = []
    y = []
    dis = []
    
    # 遍历输入文件中的每一行
    for line in lines:
        # 分割每行的数据，并处理
        attr = line.split("\n")[0].split(",")
        # 过滤掉x坐标
        if attr[1] != 'x':
            # 如果ID不在points字典中，则添加
            if attr[0] not in points:
                points[attr[0]] = [float(attr[1])]
                points[attr[0]].append(float(attr[2]))
                x.append(float(attr[1]))
                y.append(float(attr[2]))

    # 将x和y转换为NumPy数组
    x = np.array(x)
    y = np.array(y)
    # 计算x和y的均值
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    # 计算每个点到均值点的距离
    for i in range(len(x)):
        dis.append(math.pow(math.pow(x[i] - x_mean, 2) + math.pow(y[i] - y_mean, 2), 0.5))
    dis_array = np.array(dis)
    # 计算分位数和标准差
    q1 = np.percentile(dis_array, 25)
    q3 = np.percentile(dis_array, 75)
    dis_std = np.std(dis_array)
    n = len(x)  # 数据点的数量
    
    # 计算初始带宽
    delta = min(dis_std, (q3 - q1) / 1.34)
    h = 1.06 * delta * pow(len(x), -0.2)
    print("thumb-h:", h)
    
    # 初始化迭代参数
    delta_h = h / 10
    e_h = delta_h / 20
    iters = 0
    max_iters = 12
    
    # 迭代过程，寻找最优的a和h
    while (delta_a > e_a or delta_h > e_h) and max_iters > iters:
        iters += 1
        # 初始化g数组和h_list
        g = [1, 1, 1]
        h_list = [h - delta_h, h, h + delta_h]
        for i in points:
            fi = [0, 0, 0]
            # 计算每个点的KDE值
            for j in points:
                dij_square = pow(points[i][0] - points[j][0], 2) + pow(points[i][1] - points[j][1], 2)
                for h_idx in range(len(h_list)):
                    h1 = h_list[h_idx]
                    fi[h_idx] += pow(math.e, -dij_square / (2 * pow(h1, 2)))
            # 归一化KDE值
            for h_idx in range(len(h_list)):
                h1 = h_list[h_idx]
                fi[h_idx] = 1 / (2 * math.pi * n * pow(h1, 2)) * fi[h_idx]
                # 如果点的列表长度小于10，添加新的KDE值
                if len(points[i]) < 10:
                    points[i].append(fi[h_idx])
                else:
                    points[i][2 + h_idx] = fi[h_idx]
                if fi[h_idx] != 0:
                    g[h_idx] += math.log(fi[h_idx])
        
        # 更新a和计算新的h值
        a_list = [a - delta_a, a, a + delta_a]
        for h_idx in range(len(h_list)):
            g[h_idx] = pow(math.e, g[h_idx] / n)
            for i in points:
                if h_idx == 0:
                    a1 = a_list[0]
                    hi = pow(points[i][2 + h_idx] / g[h_idx], -a1) * h_list[h_idx]
                    if len(points[i]) < 10:
                        points[i].append(hi)
                    else:
                        points[i][5] = hi
                elif h_idx == 1:
                    for a_idx in range(len(a_list)):
                        a1 = a_list[a_idx]
                        hi = pow(points[i][2 + h_idx] / g[h_idx], -a1) * h_list[h_idx]
                        if len(points[i]) < 10:
                            points[i].append(hi)
                        else:
                            points[i][6 + a_idx] = hi
                else:
                    a1 = a_list[2]
                    hi = pow(points[i][2 + h_idx] / g[h_idx], -a1) * h_list[h_idx]
                    if len(points[i]) < 10:
                        points[i].append(hi)
                    else:
                        points[i][9] = hi
        
        # 计算L值和更新a和h
        L = [0, 0, 0, 0, 0]
        for i in points:
            fi = np.zeros(5)
            for j in points:
                if i != j:
                    dij_square = pow(points[i][0] - points[j][0], 2) + pow(points[i][1] - points[j][1], 2)
                    for fi_idx in range(len(fi)):
                        fi[fi_idx] += pow(math.e, -dij_square / (2 * pow(points[j][5 + fi_idx], 2))) / pow(points[j][5 + fi_idx], 2)
            for fi_idx in range(len(fi)):
                fi[fi_idx] = 1 / (2 * math.pi * (n - 1)) * fi[fi_idx]
                if fi[fi_idx] != 0:
                    L[fi_idx] += math.log(fi[fi_idx])
        
        max_L = max(L)
        if max_L == L[2]:
            delta_a = delta_a / 2
            delta_h = delta_h / 2
        else:
            idx = L.index(max_L)
            if idx == 0:
                a = a - delta_a
                h = h - delta_h
            elif idx == 1:
                a = a - delta_a
            elif idx == 3:
                a = a + delta_a
            elif idx == 4:
                a = a + delta_a
                h = h + delta_h
    
    # 记录结束时间并打印
    time_end = time.time()
    print('totally cost', time_end - time_start)
    print("the parameter of h:" + str(h) + ",the parameter of a:" + str(a))
    
    # 将最终结果写入文件
    for i in points:
        line = i + ',' + str(points[i][0]) + ',' + str(points[i][1]) + ',' + str(points[i][7]) + '\n'
        result.write(line)
    
    # 关闭结果文件
    result.close()

In [21]:
input_dir = './crime_data_2.csv'
result_dir = './adaptiveKDE_h_1000.csv'
adaptiveKDE(input_dir,result_dir)

thumb-h: 0.0031434417422994323


KeyboardInterrupt: 